In [1]:
"""Example usage:
  python object_detection/dataset_tools/create_oid_tf_record.py \
    --input_annotations_csv=/path/to/input/annotations-human-bbox.csv \
    --input_images_directory=/path/to/input/image_pixels_directory \
    --input_label_map=/path/to/input/labels_bbox_545.labelmap \
    --output_tf_record_path_prefix=/path/to/output/prefix.tfrecord
CSVs with bounding box annotations and image metadata (including the image URLs)
can be downloaded from the Open Images GitHub repository:
https://github.com/openimages/dataset
This script will include every image found in the input_images_directory in the
output TFRecord, even if the image has no corresponding bounding box annotations
in the input_annotations_csv.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import contextlib2
import pandas as pd
import tensorflow as tf

from object_detection.dataset_tools import oid_tfrecord_creation
from object_detection.utils import label_map_util

In [5]:
input_annotations_csv='/root/2017_07/validation/annotations-human-bbox.csv'
input_images_directory='/root/raw_images_validation'
input_label_map='../object_detection/data/oid_bbox_trainable_label_map.pbtxt'
!mkdir test_tfrecords
output_tf_record_path_prefix='test_tfrecords/test.tfrecord'
num_shards=100

mkdir: cannot create directory ‘test_tfrecords’: File exists


In [6]:
tf.logging.set_verbosity(tf.logging.INFO)

required_flags = [
  'input_annotations_csv', 'input_images_directory', 'input_label_map',
  'output_tf_record_path_prefix'
]

label_map = label_map_util.get_label_map_dict(input_label_map)
all_annotations = pd.read_csv(input_annotations_csv)
all_images = tf.gfile.Glob(
  os.path.join(input_images_directory, '*.jpg'))
all_image_ids = [os.path.splitext(os.path.basename(v))[0] for v in all_images]
all_image_ids = pd.DataFrame({'ImageID': all_image_ids})
all_annotations = pd.concat([all_annotations, all_image_ids])

tf.logging.log(tf.logging.INFO, 'Found %d images...', len(all_image_ids))

INFO:tensorflow:Found 41620 images...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
